In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [50]:
image_size = [512, 512] # Default Image size

train_path = r"/content/gdrive/My Drive/Deep Learning/Datasets/dogs_vs_cats/train/train/"
valid_path = r"/content/gdrive/My Drive/Deep Learning/Datasets/dogs_vs_cats/train/valid/"

test_path = r"/content/gdrive/My Drive/Deep Learning/Datasets/dogs_vs_cats/train/test/"

# Pre-Trained Model - Resnet50

In [51]:
resnet = ResNet50(input_shape= image_size+[3],
                  weights = 'imagenet',
                  include_top= False)
# include_top is to remove resnet last layer so we use our custom output layer, resnet has abt 1000 output neurons we need only 2

In [52]:
# Not training all layers but we want to use pretrained weights also
for layer in resnet.layers:
  layer.trainable = False

Building the remaining model architecture
- We are creating our model using resnets pretrained weights

In [53]:
# My own layers - I can add more if you want
x = Flatten()(resnet.output)

In [54]:
# Our custom output layer
prediction = Dense(2, activation='softmax')(x)

In [55]:
## Create a model object
model = Model(inputs = resnet.input, outputs=prediction)

In [56]:
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 256, 256, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 256, 256, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [61]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics = ['accuracy'])

# Data Augementation

In [17]:
train_datagen = ImageDataGenerator(preprocessing_function= preprocess_input,
                                   rescale = 1/255.0,
                                   shear_range= 0.2,
                                   zoom_range= 0.2,
                                   horizontal_flip= True)

valid_datagen = ImageDataGenerator(preprocessing_function= preprocess_input,
                                   rescale= 1/255.0)

In [64]:
%%time
X_train = train_datagen.flow_from_directory(train_path,
                                            target_size = (512, 512),
                                            batch_size= 32,
                                            class_mode= 'categorical')

X_valid = valid_datagen.flow_from_directory(valid_path,
                                            target_size = (512, 512),
                                            batch_size= 32,
                                            class_mode= 'categorical')

Found 1000 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
CPU times: user 28.1 ms, sys: 1.05 ms, total: 29.2 ms
Wall time: 210 ms


Checking for classes

In [25]:
X_train.class_indices

{'cat': 0, 'dog': 1}

In [26]:
X_valid.class_indices

{'cat': 0, 'dog': 1}

# Let train

In [27]:
from  tensorflow.keras.callbacks import EarlyStopping

In [62]:
early_stop = EarlyStopping(monitor='val_loss',
                           patience= 7,
                           mode= 'min',
                           restore_best_weights = True)

In [67]:
%%time
model.fit_generator(X_train,
                    validation_data= X_valid,
                    epochs=20,
                    callbacks=[early_stop])

Epoch 1/20
32/32 [==============================] - 56s 2s/step - loss: 6.1573 - accuracy: 0.6150 - val_loss: 5.3506 - val_accuracy: 0.6600
Epoch 2/20
32/32 [==============================] - 56s 2s/step - loss: 7.6295 - accuracy: 0.6180 - val_loss: 2.5800 - val_accuracy: 0.7450
Epoch 3/20
32/32 [==============================] - 55s 2s/step - loss: 9.7177 - accuracy: 0.6230 - val_loss: 35.2239 - val_accuracy: 0.5000
Epoch 4/20
32/32 [==============================] - 55s 2s/step - loss: 24.3349 - accuracy: 0.5670 - val_loss: 61.5714 - val_accuracy: 0.5000
Epoch 5/20
32/32 [==============================] - 55s 2s/step - loss: 25.1090 - accuracy: 0.5630 - val_loss: 17.6118 - val_accuracy: 0.5150
Epoch 6/20
32/32 [==============================] - 55s 2s/step - loss: 14.1223 - accuracy: 0.6110 - val_loss: 5.6613 - val_accuracy: 0.7350
Epoch 7/20
32/32 [==============================] - 55s 2s/step - loss: 6.0229 - accuracy: 0.6670 - val_loss: 12.6530 - val_accuracy: 0.5900
Epoch 8/20
32

To save us from retraining let save this model

In [30]:
from tensorflow.keras.models import load_model
import pandas as pd

In [68]:
losses = pd.DataFrame(model.history.history)
losses

,loss,accuracy,val_loss,val_accuracy
0,6.157308,0.615,5.350557,0.660
1,7.629517,0.618,2.579954,0.745
2,9.717726,0.623,35.223942,0.500
3,24.334904,0.567,61.571365,0.500
4,25.108961,0.563,17.611782,0.515
5,14.122281,0.611,5.661321,0.735
6,6.022950,0.667,12.653028,0.590
7,10.418480,0.633,11.995227,0.545
8,6.593717,0.660,15.108789,0.525


In [31]:
model.save('/content/gdrive/My Drive/Deep Learning/Datasets/dogs_vs_cats/dogs_vs_cats.h5')

In [32]:
model.save_weights('/content/gdrive/My Drive/Deep Learning/Datasets/dogs_vs_cats/dogs_vs_cats_weights.h5')

# Convert to json file for ease of sharing/production

In [34]:
model_json = model.to_json()
with open('/content/gdrive/My Drive/Deep Learning/Datasets/dogs_vs_cats/dogs_vs_cats_model.json', 'w') as json_file:
  json_file.write(model_json)

,loss,accuracy,val_loss,val_accuracy
0,2.518196,0.525,1.299890,0.500
1,0.958788,0.559,0.651320,0.680
2,0.795026,0.561,0.840068,0.545
3,0.847150,0.597,0.810342,0.570
4,0.783405,0.596,0.776442,0.570
5,1.043322,0.545,0.999584,0.540
6,0.998153,0.604,1.496563,0.515
